In [11]:
import pandas as pd

In [19]:
df_noticias = pd.read_csv('data/df_all_news.csv')
df_tweets = pd.read_csv('data/tweets.csv')

In [24]:
df_noticias.head()

,title,subtitle,content,published_date,link,address_types,coordinates,site,search_term
0,Centro-oeste paulista registrou mais de 8 mil ...,"Marília, Bauru, Lençóis Paulista, Assis e Botu...","O centro-oeste paulista registrou, em 2024, ma...",24/04/2025,https://g1.globo.com/sp/bauru-marilia/noticia/...,[],[],g1,acidente
1,Defesa Civil promove oficina para preparar pro...,Capacitação faz parte da Operação São Paulo Se...,A Defesa Civil do Estado de São Paulo realiza ...,23/04/2025,https://g1.globo.com/sp/bauru-marilia/noticia/...,['Rua'],"[(-22.3145563, -49.07876599999999)]",g1,operac
2,Justiça arquiva inquérito que investigava prof...,Manifestações referentes à conduta do ex-docen...,A Justiça arquivou o inquérito que investigava...,23/04/2025,https://g1.globo.com/sp/bauru-marilia/noticia/...,[],[],g1,suspeit
3,Visita do papa Francisco ao Brasil em 2013 ins...,"Padre Murilo Bernardes, de Bauru (SP), encontr...",Erro ao recuperar o conteúdo,22/04/2025,https://g1.globo.com/sp/bauru-marilia/noticia/...,[],[],g1,morre
4,Igrejas realizam missas e celebrações especiai...,Pontífice morreu aos 88 anos nesta segunda-fei...,Igrejas das regiões de Bauru e Marília (SP) re...,21/04/2025,https://g1.globo.com/sp/bauru-marilia/noticia/...,"['Vila', 'Praça', 'Rua']","[(-22.3227925, -49.0730045), (-22.3155297, -49...",g1,morre


In [14]:
df_tweets.head()

,tweet_count,username,text,created_at,retweets,likes
1,tadssss,toda vez q alguem vem me falar q ta triste pq ...,Fri Mar 14 17:57:39 +0000 2025,0,16,morre
2,Gabiiii,"odeio qdo acaba luz em Bauru, privatizaram ess...",Sun Mar 16 00:18:07 +0000 2025,0,1,"cpfl, energia, luz, prefeitura, árvore"
3,faby jurídico modern family,SAMU: 192\nbombeiros: 193\npolícia federal: 19...,Wed Mar 20 02:17:55 +0000 2024,7367,48391,"bombeiro, delegacia, morre, polícia, samu"
4,Governo de S. Paulo,Antes e depois da Rodovia Osni Mateus (SP 261)...,Tue Apr 15 22:23:09 +0000 2025,6,23,obra
5,Ana Bia,Qualquer poça de água é alagamento aqui em Bauru,Wed Mar 26 10:14:01 +0000 2025,0,0,"alagamento, água"
